In [88]:
import tensorflow as tf
from tensorflow import keras 

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import matplotlib.pyplot as plt




### Create df

In [89]:
# define both features and labels dataframes, save as original dfs

df_ori_features = pd.read_csv('heart-disease-values.csv')

df_ori_labels = pd.read_csv('heart-disease-labels.csv')


# define two dfs which we will manipulate

df_features = df_ori_features

df_labels = df_ori_labels

In [90]:
# merge both dfs (NB we don't need to do this to feed X and y into our model. 
# But it's good to know how to.)



In [91]:
df_labels

,patient_id,heart_disease_present
0,0z64un,0
1,ryoo3j,0
2,yt1s1x,1
3,l2xjde,1
4,oyt4ek,0
...,...,...
175,5qfar3,1
176,2s2b1f,1
177,nsd00i,1
178,0xw93k,0


### define X and y

In [92]:
# define X and y

X = df_features.drop(['patient_id'], axis='columns') # drop patient_id col

y = df_labels.drop(columns=['patient_id']) # drop patient_id col

In [93]:
# inspect df
    
X

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,1,normal,128,2,0,0,2,308,0.0,1,45,170,0
1,2,normal,110,3,0,0,0,214,1.6,0,54,158,0
2,1,normal,125,4,3,0,2,304,0.0,1,77,162,1
3,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0
4,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2,reversible_defect,125,4,2,1,0,254,0.2,1,67,163,0
176,2,normal,180,4,0,0,1,327,3.4,0,55,117,1
177,2,reversible_defect,125,3,0,0,0,309,1.8,1,64,131,1
178,1,normal,124,3,2,1,0,255,0.0,1,48,175,0


In [94]:
# inspect df    

y 

# seeing that y has 1 or 0 (integers), label encoding won't be necessary

,heart_disease_present
0,0
1,0
2,1
3,1
4,0
...,...
175,1
176,1
177,1
178,0


In [95]:
# inspect data some more, how'as the distribution? Influences the feature scaling I'll use

X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
slope_of_peak_exercise_st_segment,180.0,1.550000,0.618838,1.0,1.00,1.0,2.00,3.0
resting_blood_pressure,180.0,131.311111,17.010443,94.0,120.00,130.0,140.00,180.0
chest_pain_type,180.0,3.155556,0.938454,1.0,3.00,3.0,4.00,4.0
num_major_vessels,180.0,0.694444,0.969347,0.0,0.00,0.0,1.00,3.0
fasting_blood_sugar_gt_120_mg_per_dl,180.0,0.161111,0.368659,0.0,0.00,0.0,0.00,1.0
resting_ekg_results,180.0,1.050000,0.998742,0.0,0.00,2.0,2.00,2.0
serum_cholesterol_mg_per_dl,180.0,249.211111,52.717969,126.0,213.75,245.5,281.25,564.0
oldpeak_eq_st_depression,180.0,1.010000,1.121357,0.0,0.00,0.8,1.60,6.2
sex,180.0,0.688889,0.464239,0.0,0.00,1.0,1.00,1.0
age,180.0,54.811111,9.334737,29.0,48.00,55.0,62.00,77.0


### remove outliers

In [96]:
# serum_cholesterol_mg_per_dl	180.0	249.211111	52.717969	126.0	213.75	245.5	281.25	564.0
# max is so much higher than 75% (upper quartile) -- could this indicate many outliers in this row?

X['serum_cholesterol_mg_per_dl']

# drop rows 3 std devs from the mean (std = 53, mean = 249, 3 std devs = 159, 
# therefore anything over 408 and under 90 is an outlier)


0      308
1      214
2      304
3      223
4      270
      ... 
175    254
176    327
177    309
178    255
179    201
Name: serum_cholesterol_mg_per_dl, Length: 180, dtype: int64

In [97]:
# get index of all rows in 'serum_cholesterol_mg_per_dl' > 408 


condition1 = X['serum_cholesterol_mg_per_dl'] > 408

condition1_idx = X.index[condition1]

list_over_408 = condition1_idx.to_list()

print(list_over_408)


# get index of all rows in 'serum_cholesterol_mg_per_dl' < 90

condition2 = X['serum_cholesterol_mg_per_dl'] < 90

condition2_idx = X.index[condition2]

list_under_90 = condition2_idx.to_list()

print(list_under_90) # no values under 90, cf describe().transpose() above, min = 126



[43, 60]
[]


In [102]:
# remove rows on index position [43, 60]

X = X.drop(list_over_408) 

In [103]:
# inspect X without outliers

X

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,1,normal,128,2,0,0,2,308,0.0,1,45,170,0
1,2,normal,110,3,0,0,0,214,1.6,0,54,158,0
2,1,normal,125,4,3,0,2,304,0.0,1,77,162,1
3,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0
4,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2,reversible_defect,125,4,2,1,0,254,0.2,1,67,163,0
176,2,normal,180,4,0,0,1,327,3.4,0,55,117,1
177,2,reversible_defect,125,3,0,0,0,309,1.8,1,64,131,1
178,1,normal,124,3,2,1,0,255,0.0,1,48,175,0


### feature scale the values